In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import midynet
import seaborn as sb
import os

from collections import defaultdict

## Conversion of Figure 2 data

This data has been available on narval.

In [45]:
dynamics = ["glauber", "sis", "cowan"]
acceptables = ["prior", "posterior", "evidence", "likelihood", "mutualinfo"]

all_df = {}
for d in dynamics:
    og_data = pd.read_pickle(f"data/archive/figure2/exact-{d}/recon_information.pickle")
    coupling = np.array([0.25, 0.5, 1])
    if d == "glauber":
        clabel = "coupling"
    elif d == "sis":
        clabel = "infection_prob"
    elif d == "cowan":
        clabel = "nu"
    num_steps = np.unique(np.logspace(1, 4, 100).astype("int"))
    J, T = np.meshgrid(coupling, num_steps)
    
    data = defaultdict(list)
    for i, (j, t) in enumerate(zip(J.flatten(), T.flatten())):
        data[f"data_model.{clabel}"].append(j)
        data["data_model.length"].append(t)

    for k, v in og_data[f"figure2-exact-{d}"].items():
        key, agg = k.split("-")
        if key not in acceptables:
            continue
        for i, m in enumerate(v.flatten()):
            if agg == "mid":
                data[key + "-loc"].append(m)
            elif len(data[key + "-loc"]) > len(data[key + "-scale"]):
                data[key + "-scale"].append(m)
            else:
                data[key + "-scale"][-1] = 0.5 * (data[key + "-scale"][-1] + m)
    df = pd.DataFrame(data)
    mi = midynet.statistics.Statistics.from_dataframe(df, key="mutualinfo")
    prior = midynet.statistics.Statistics.from_dataframe(df, key="prior")
    evidence = midynet.statistics.Statistics.from_dataframe(df, key="evidence")
    data["recon-loc"], data["recon-scale"] = (mi/prior).loc, (mi/prior).scale
    data["pred-loc"], data["pred-scale"] = (mi/evidence).loc, (mi/evidence).scale
    df = pd.DataFrame({f"exact-{d}":data})
    
    path = f"data/figure2/exact-{d}"
    os.makedirs(path, exist_ok=True)
    df.to_pickle(os.path.join(path, "reconinfo.pkl"))
    

## Conversion of Figure 3 data

Original data has been generated on graham.

In [108]:


coupling = [
    np.unique(np.concatenate([
        np.linspace(0, 0.2, 10), np.linspace(0.2, 0.8, 15)
    ])),
    np.array([0.0, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]),
]
coupling[1] /= 5
method = ["meanfield", "annealed"]
name_map = {
    "hg": "prior",
    "hx": "evidence",
    "hxg": "likelihood",
    "hgx": "posterior",
    "mi": "mutualinfo",
}

for m, c in zip(method, coupling):
    data = defaultdict(list)
    data["data_model.coupling"] = c
    og_data = pd.read_pickle(f"data/archive/figure3/figure3-large-T1000-ising.{m}/mutualinfo.pickle")
    for k, v in og_data[f"figure3-large-T1000-ising.{m}"].items():
        key, agg = k.split("-")
        key = name_map[key]
        if agg == "mid":
            data[key + "-loc"] = v
        elif len(data[key + "-loc"]) > len(data[key + "-scale"]):
            data[key + "-scale"] = v
        else:
            data[key + "-scale"] = 0.5 * (data[key + "-scale"] + v)
    df = pd.DataFrame(data)
    mi = midynet.statistics.Statistics.from_dataframe(df, key="mutualinfo")
    prior = midynet.statistics.Statistics.from_dataframe(df, key="prior")
    evidence = midynet.statistics.Statistics.from_dataframe(df, key="evidence")
    data["recon-loc"], data["recon-scale"] = (mi/prior).loc, (mi/prior).scale
    data["pred-loc"], data["pred-scale"] = (mi/evidence).loc, (mi/evidence).scale
    df = pd.DataFrame(data)
    p = f"data/figure3/{m}-glauber-large"
    os.makedirs(p, exist_ok=True)
    pd.to_pickle({f"glauber-large-{m}": df}, os.path.join(p, "reconinfo.pkl"))
    
    

## Conversion of Figure 4 data
Original data has been generated narval.

In [121]:
og_data = pd.read_pickle("data/archive/figure4/figure4-large-nbinom-ising/mutualinfo.pickle")
dynamics = ["glauber", "sis", "cowanfw", "cowanbw"]

dmap = {
    "glauber": "ising" ,
    "sis": "sis" ,
    "cowanfw": "cowan_forward" ,
    "cowanbw": "cowan_backward" ,
}
name_map = {
    "hg": "prior",
    "hx": "evidence",
    "hxg": "likelihood",
    "hgx": "posterior",
    "mi": "mutualinfo",
}
clabel = {
    "glauber": "coupling",
    "sis": "infection_prob",
    "cowanfw": "nu",
    "cowanbw": "nu",
}

coupling = {
    "glauber": np.concatenate([np.linspace(0., 0.5, 6), np.linspace(0.5, 0.75, 22), np.linspace(0.75, 1.5, 6)])/5,
    "sis": np.concatenate([np.linspace(0., 0.125, 3), np.linspace(0.125, 0.5, 30)])/5,
    "cowanfw": np.concatenate([np.linspace(2., 2.5, 23),np.linspace(2.5, 4, 10),])/5,
    "cowanbw": np.concatenate([np.linspace(1.0, 1.25, 4),np.linspace(1.25, 1.5, 20), np.linspace(1.5, 3, 10)])/5,
    
}

for d in dynamics:
    data = defaultdict(list)
    data[f"data_model.{clabel[d]}"] = np.unique(coupling[d])
    og_data = pd.read_pickle(f"data/archive/figure4/figure4-large-nbinom-{dmap[d]}/mutualinfo.pickle")
    for k, v in og_data[f"figure4-large-nbinom-{dmap[d]}"].items():
        key, agg = k.split("-")
        key = name_map[key]
        if agg == "mid":
            data[key + "-loc"] = v
        elif len(data[key + "-loc"]) > len(data[key + "-scale"]):
            data[key + "-scale"] = v
        else:
            data[key + "-scale"] = 0.5 * (data[key + "-scale"] + v)
    df = pd.DataFrame(data)
    mi = midynet.statistics.Statistics.from_dataframe(df, key="mutualinfo")
    prior = midynet.statistics.Statistics.from_dataframe(df, key="prior")
    evidence = midynet.statistics.Statistics.from_dataframe(df, key="evidence")
    data["recon-loc"], data["recon-scale"] = (mi/prior).loc, (mi/prior).scale
    data["pred-loc"], data["pred-scale"] = (mi/evidence).loc, (mi/evidence).scale
    df = pd.DataFrame(data)
    p = f"data/figure4/geometric-{d}"
    os.makedirs(p, exist_ok=True)
    pd.to_pickle({f"geometric-{d}": df}, os.path.join(p, "reconinfo.pkl"))
        

In [120]:
df

,data_model.nu,evidence-loc,evidence-scale,prior-loc,prior-scale,likelihood-loc,likelihood-scale,posterior-loc,posterior-scale,mutualinfo-loc,mutualinfo-scale,recon-loc,recon-scale,pred-loc,pred-scale
0,0.200000,1.196508e+04,92.158943,17999.974546,1.206128,1.321957e+04,89.073658,19230.325689,10.164619,-1222.563395,10.864619,-0.067920,0.000608,-0.102178,0.001695
1,0.216667,1.260812e+04,78.878858,17999.974546,1.206128,1.372165e+04,82.531596,19109.232908,8.684911,-1114.123191,7.853372,-0.061896,0.000440,-0.088365,0.001176
2,0.233333,1.368212e+04,104.219339,17999.974546,1.206128,1.456400e+04,100.180548,18941.345721,13.075754,-939.765454,13.556499,-0.052209,0.000757,-0.068686,0.001514
3,0.250000,1.617366e+04,173.681576,17999.974546,1.206128,1.676278e+04,148.204445,18573.401998,28.076140,-568.634970,28.924260,-0.031591,0.001609,-0.035158,0.002166
4,0.252632,1.686447e+04,198.743336,17999.974546,1.206128,1.729024e+04,176.811946,18461.935069,25.949129,-463.960523,26.014275,-0.025776,0.001447,-0.027511,0.001867
5,0.255263,1.804817e+04,258.154764,17999.974546,1.206128,1.845158e+04,230.433406,18253.048186,40.658471,-252.073639,41.214570,-0.014004,0.002291,-0.013967,0.002483
6,0.257895,1.945577e+04,549.257846,17999.974546,1.206128,1.958614e+04,494.735228,18060.963587,69.220346,-53.696560,69.375893,-0.002983,0.003854,-0.002760,0.003644
7,0.260526,2.157616e+04,855.977571,17999.974546,1.206128,2.132338e+04,737.801751,17749.197127,117.820840,252.777419,118.645840,0.014043,0.006591,0.011716,0.005034
8,0.263158,2.705103e+04,3645.907787,17999.974546,1.206128,2.611426e+04,3230.289715,17063.999103,405.887601,936.767924,406.066849,0.052043,0.022556,0.034630,0.010344
9,0.265789,5.508777e+04,9173.515525,17999.974546,1.206128,5.162145e+04,8550.698130,14537.659083,625.819426,3466.315463,626.269426,0.192573,0.034780,0.062924,0.000890
